<center><h1>Last_First_HW3</h1></center>
<br>
<br>

Name: Yu-Hsuan Lu
<br>
Github Username: @YuHsuan-Lu
<br>
USC ID: 6849388612

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [685]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import csv
from sklearn.utils import resample

Get the AReM Data Set

In [686]:
test = []
train = []
# Specify the parent folder path
root = '../data/AReM'

# List the items (files and directories) in the parent folder
folders = os.listdir(root)

# Iterate through the items and filter out directories
for f in folders:
    f_path = os.path.join(root,f)  # Full path to the item   
    # Check if the item is a directory
    if os.path.isdir(f_path):
        if f == 'bending1' or f == 'bending2':
            for file in os.listdir(f_path):
                file_path = os.path.join(f_path,file)
                if file == 'dataset1.csv' or file == 'dataset2.csv':
                    test.append(file_path)
                else:
                    train.append(file_path)
        else:
            for file in os.listdir(f_path):
                file_path = os.path.join(f_path,file)
                if file == 'dataset1.csv' or file == 'dataset2.csv'or file == 'dataset3.csv':
                    test.append(file_path)
                else:
                    train.append(file_path)

### (b) Test and Train Data

In [687]:
test

['../data/AReM/bending1/dataset1.csv',
 '../data/AReM/bending1/dataset2.csv',
 '../data/AReM/walking/dataset1.csv',
 '../data/AReM/walking/dataset2.csv',
 '../data/AReM/walking/dataset3.csv',
 '../data/AReM/bending2/dataset1.csv',
 '../data/AReM/bending2/dataset2.csv',
 '../data/AReM/standing/dataset1.csv',
 '../data/AReM/standing/dataset2.csv',
 '../data/AReM/standing/dataset3.csv',
 '../data/AReM/sitting/dataset1.csv',
 '../data/AReM/sitting/dataset2.csv',
 '../data/AReM/sitting/dataset3.csv',
 '../data/AReM/lying/dataset1.csv',
 '../data/AReM/lying/dataset2.csv',
 '../data/AReM/lying/dataset3.csv',
 '../data/AReM/cycling/dataset1.csv',
 '../data/AReM/cycling/dataset2.csv',
 '../data/AReM/cycling/dataset3.csv']

In [688]:
train

['../data/AReM/bending1/dataset7.csv',
 '../data/AReM/bending1/dataset6.csv',
 '../data/AReM/bending1/dataset4.csv',
 '../data/AReM/bending1/dataset5.csv',
 '../data/AReM/bending1/dataset3.csv',
 '../data/AReM/walking/dataset7.csv',
 '../data/AReM/walking/dataset6.csv',
 '../data/AReM/walking/dataset4.csv',
 '../data/AReM/walking/dataset5.csv',
 '../data/AReM/walking/dataset10.csv',
 '../data/AReM/walking/dataset11.csv',
 '../data/AReM/walking/dataset13.csv',
 '../data/AReM/walking/dataset12.csv',
 '../data/AReM/walking/dataset15.csv',
 '../data/AReM/walking/dataset14.csv',
 '../data/AReM/walking/dataset8.csv',
 '../data/AReM/walking/dataset9.csv',
 '../data/AReM/bending2/dataset6.csv',
 '../data/AReM/bending2/dataset4.csv',
 '../data/AReM/bending2/dataset5.csv',
 '../data/AReM/bending2/dataset3.csv',
 '../data/AReM/standing/dataset7.csv',
 '../data/AReM/standing/dataset6.csv',
 '../data/AReM/standing/dataset4.csv',
 '../data/AReM/standing/dataset5.csv',
 '../data/AReM/standing/dataset

### (c) Feature Extraction

#### i. Research

#### ii. Extraction

In [689]:
COLUMN_NAMES = ["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"]
STATS = ['min', 'max', 'mean', 'median', 'std', '1st_quartile', '3rd_quartile']

In [690]:
def load_and_get_sst(files, feature, cols):    
    instances  = []
    labels = []
    for f in files:
        move_label = os.path.basename(os.path.dirname(f))
        labels.append(move_label)
        
        delimiter = csv.Sniffer().sniff(open(f).read()).delimiter
        if delimiter == ",":
            fc = pd.read_csv(f, skiprows=5, header=None, on_bad_lines='skip')
        else:
            fc = pd.read_csv(f, skiprows=5, sep="\\s+", header=None, on_bad_lines='skip')
            
        fc.columns = COLUMN_NAMES

        sst = []
        
        cur_seg = fc[0 : fc.shape[0]]
        cur_describe = cur_seg.describe().drop('count').drop(columns="# Columns: time").T
        sst.append(cur_describe.values.flatten())

        instances.append(np.concatenate(sst))
            
    
    sst_df = pd.DataFrame(instances, columns=get_columns())[feature]
    sst_df['label'] = pd.Series(labels)
    
    return sst_df


feature = [f'{stat}{i}' for i in range(1, 7) for stat in STATS]
train_data = load_and_get_sst(train, feature, COLUMN_NAMES)
test_data = load_and_get_sst(test, feature, COLUMN_NAMES)

train_data

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6,label
0,43.969125,1.618364,36.25,43.310,44.50,44.67,48.00,0.413125,0.263111,0.0,...,23.7500,30.75,0.555312,0.487826,0.0,0.0000,0.490,0.830,2.96,bending1
1,43.454958,1.386098,37.00,42.500,43.25,45.00,48.00,0.378083,0.315566,0.0,...,24.0000,33.50,0.679646,0.622534,0.0,0.4300,0.500,0.870,5.26,bending1
2,42.179812,3.670666,33.00,39.150,43.50,45.00,47.75,0.696042,0.630860,0.0,...,36.3300,38.67,0.613521,0.524317,0.0,0.0000,0.500,1.000,2.18,bending1
3,41.678063,2.243490,33.00,41.330,41.75,42.75,45.75,0.535979,0.405469,0.0,...,31.2500,37.50,0.383292,0.389164,0.0,0.0000,0.430,0.500,1.79,bending1
4,43.954500,1.558835,35.00,43.000,44.33,45.00,47.40,0.426250,0.338690,0.0,...,36.5000,38.50,0.493292,0.513506,0.0,0.0000,0.430,0.940,1.79,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,35.752354,4.614802,18.50,33.000,36.00,39.33,44.25,3.328104,2.140576,0.0,...,18.0625,24.33,3.069667,1.748326,0.0,1.7975,2.770,4.060,9.39,cycling
65,37.177042,3.581301,24.25,34.500,36.25,40.25,45.00,2.374208,1.601799,0.0,...,21.7500,25.50,2.921729,1.852600,0.0,1.5000,2.500,3.900,9.34,cycling
66,36.248768,3.824632,23.33,33.415,36.75,39.25,43.50,2.737307,2.093999,0.0,...,21.0000,27.00,3.532463,1.965267,0.0,2.1700,3.110,4.625,11.15,cycling
67,36.957458,3.434863,26.25,34.500,36.29,40.25,44.25,2.420083,1.724901,0.0,...,18.2500,22.25,2.934625,1.631380,0.0,1.6600,2.525,4.030,8.34,cycling


#### iii. Standard Deviation

In [691]:
###generate statistics for all columns
#train_data.describe()
###only select the std data for all columns
train_data.describe().loc['std']

min1             4.917692
max1             1.758670
mean1            8.794295
median1          5.731647
std1             4.956111
1st_quartile1    4.783645
3rd_quartile1    4.429182
min2             1.600701
max2             0.902808
mean2            0.000000
median2          0.952201
std2             1.436960
1st_quartile2    2.158416
3rd_quartile2    5.147841
min3             3.863097
max3             0.995959
mean3            3.053869
median3          4.145255
std3             3.845730
1st_quartile3    3.946023
3rd_quartile3    4.759853
min4             1.179861
max4             0.473576
mean4            0.000000
median4          0.842501
std4             1.150092
1st_quartile4    1.566564
3rd_quartile4    2.302408
min5             5.120426
max5             1.057998
mean5            5.368786
median5          5.543882
std5             5.267414
1st_quartile5    4.957231
3rd_quartile5    5.449726
min6             1.171401
max6             0.519420
mean6            0.051766
median6     

In [692]:
###generate statistics for all columns
#test_data.describe()
###only select the std data for all columns
test_data.describe().loc['std']

min1              6.790086
max1              1.869285
mean1            12.136206
median1           7.673052
std1              7.088085
1st_quartile1     6.385720
3rd_quartile1     4.379342
min2              1.500529
max2              0.832472
mean2             0.000000
median2           0.941307
std2              1.345903
1st_quartile2     2.028482
3rd_quartile2     4.870395
min3              4.588252
max3              0.760779
mean3             2.644618
median3           4.554121
std3              4.753522
1st_quartile3     5.014339
3rd_quartile3     5.393220
min4              1.146338
max4              0.409008
mean4             0.000000
median4           0.867088
std4              1.161705
1st_quartile4     1.541943
3rd_quartile4     1.733937
min5              7.366781
max5              0.919274
mean5             8.252947
median5           7.823870
std5              7.438749
1st_quartile5     7.259433
3rd_quartile5     6.782153
min6              1.119410
max6              0.525263
m

#### iv. Select Features

In [693]:
n_iterations = 10000# Number of bootstrap iterations

data = train_data.iloc[:, :-1]

bootstrap_samples = [resample(data).apply(lambda col: np.std(col)) for _ in range(n_iterations)]
    
    
bootstrap_samples_df = pd.DataFrame(bootstrap_samples, columns=list(data.columns))

# Calculate the 5th and 95th percentiles for each column
interval = bootstrap_samples_df.apply(lambda col : (np.percentile(col, 5), np.percentile(col, 95)), axis=0)

# Round the values to 2 decimal places
interval = interval.round(2)

interval.insert(loc=0, column='bound', value=['low','high'])

interval



,bound,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,...,std5,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6
0,low,4.29,1.53,7.48,5.11,4.29,3.9,3.25,1.39,0.80,...,3.70,3.56,4.26,1.06,0.47,0.00,0.69,0.99,1.40,2.21
1,high,5.40,1.93,9.98,6.17,5.46,5.5,5.36,1.72,0.96,...,6.49,6.06,6.34,1.23,0.55,0.09,0.82,1.16,1.62,2.80


In [694]:
width = [0] * len(STATS)
cols = interval.columns
for i in range(len(STATS)):
    for c in cols:
        if ORDERED_STATS[i] in c:
            width[i] += interval[c].iloc[1] - interval[c].iloc[0]
width
            

[5.62,
 1.6199999999999999,
 6.369999999999999,
 5.489999999999999,
 5.85,
 6.199999999999999,
 7.43]

since when i=2,4,5 <br>
interval has the smallest width, which is helpful for classfication <br>
which means stats[2],stats[4],stats[5] are the most important 3 features: <br>
 <b>'mean', 'std', '1st_quartile'</b><br>

## 2. ISLR 3.7.4

### (a) Linear Train

### (b) Linear Test

### (c) Not Linear Train

### (d) Not Linear Testing

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 